In [3]:
import os
import sys
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch
import matplotlib.pyplot as plt
from Naive_DAE import Naive_DAE,Dropout_DAE
import AE_Stats
from load_data_fn import load_data, load_data_non_norm

from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, TensorDataset
import importlib


In [4]:
dt,dt_loc = load_data_non_norm(1)

In [5]:
wafer_energy = torch.sum(dt[:,0:48],dim =1)

In [7]:
# dt_raw = torch.clone(dt)

In [10]:
dt = dt/ torch.unsqueeze(wafer_energy,dim=1)

In [29]:
filtered_dt = dt[~torch.any(dt.isnan(),dim=1)]


tensor(1.)

In [32]:
size_train =50000
size_test = 10000
train_loc = dt_loc[0:size_train]
test_loc = dt_loc[-size_test:]
# train = torch.vstack([train,largest[0:1000]] )
train_loc=train_loc[torch.randperm(train_loc.size()[0])]
test_loc=test_loc[torch.randperm(test_loc.size()[0])]
train_loc_dl_flat = DataLoader(
    TensorDataset(torch.Tensor(train_loc)),
    batch_size=100,
    shuffle=False
)

test_loc_dl_flat = DataLoader(
    TensorDataset(torch.Tensor(test_loc)),
    batch_size=100,
    shuffle=False
)

train = filtered_dt[0:size_train]
test = filtered_dt[-size_test:]
# train = torch.vstack([train,largest[0:1000]] )
train=train[torch.randperm(train.size()[0])]
test=test[torch.randperm(test.size()[0])]
train_dl_flat = DataLoader(
    TensorDataset(torch.Tensor(train)),
    batch_size=100,
    shuffle=False
)

test_dl_flat = DataLoader(
    TensorDataset(torch.Tensor(test)),
    batch_size=100,
    shuffle=False
)

In [34]:
def AE_MSE(data,pred):
    return torch.mean(torch.sum((data-pred)**2,dim=1))

def AE_MAE(data,pred):
    return torch.mean(torch.sum(torch.abs(data-pred),dim=1))

def weighted_AE_MSE(data, pred):
    return torch.mean(torch.sum((data[:,0:48])**2,dim=1)*torch.sum((data-pred)**2,dim=1))

def dif_weighted_AE_MSE(dat`a, pred,alpha = 1):
    return torch.mean(((torch.sum(data[:,0:48]**2,dim=1)**0.5)*alpha+1)*(torch.sum((data-pred)**2,dim=1)))

def log_dif_weighted_AE_MSE(data, pred,alpha = 1):
    return torch.mean(torch.log((torch.sum(data[:,0:48]**2,dim=1)**0.5)*alpha+1)*(torch.sum((data-pred)**2,dim=1)))

In [42]:
num_epochs = 100
lr = 0.0045
loss = log_dif_weighted_AE_MSE

In [43]:
# fine-tune autoencoder
#batch 500


model_5 = Naive_DAE([48,100,50,20,16])

optimizer = optim.Adam(model_5.parameters(), lr,weight_decay=5e-2)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor = 0.5)

all_test_losses = []
all_train_losses = []
# train
running_loss = float("inf")
for epoch in range(num_epochs):
    
    losses = []
    
    for i, data_list in enumerate(train_dl_flat):
        model_5.train()
        data = data_list[0]
        v_pred = model_5(data)
        
        batch_loss = loss(data, v_pred) # difference between actual and reconstructed   
        
        
        all_train_losses.append(batch_loss.item())
        losses.append(batch_loss.item())
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
    scheduler.step(batch_loss)
    data_test = test
    model_5.eval()
    test_pred = model_5(data_test)
    batch_test = loss(data_test, test_pred)
    running_loss = np.mean(losses)
    running_test_loss = batch_test.item()
    print('Epoch {}, lr {}'.format(
        epoch, optimizer.param_groups[0]['lr']))
    
    print(f"Epoch {epoch}: Train {running_loss}, Test {running_test_loss}")
model_5.eval()
AE_Stats.gen_all_stats(model_5(test),test,test_loc)


Epoch 0, lr 0.0045
Epoch 0: Train 0.25754720261693, Test 0.2573533058166504
Epoch 1, lr 0.0045
Epoch 1: Train 0.2570022059082985, Test 0.2573603689670563
Epoch 2, lr 0.0045
Epoch 2: Train 0.2570104503035545, Test 0.25736328959465027
Epoch 3, lr 0.0045
Epoch 3: Train 0.2570136491954327, Test 0.2573646306991577
Epoch 4, lr 0.0045
Epoch 4: Train 0.2570151609480381, Test 0.25736528635025024
Epoch 5, lr 0.0045
Epoch 5: Train 0.2570159243345261, Test 0.2573656141757965
Epoch 6, lr 0.0045
Epoch 6: Train 0.2570163213908672, Test 0.2573658525943756
Epoch 7, lr 0.0045
Epoch 7: Train 0.2570163907408714, Test 0.257365882396698
Epoch 8, lr 0.0045
Epoch 8: Train 0.2570163444876671, Test 0.25736546516418457
Epoch 9, lr 0.0045
Epoch 9: Train 0.25701648420095446, Test 0.2573659121990204
Epoch 10, lr 0.0045
Epoch 10: Train 0.25701639932394027, Test 0.2573659121990204
Epoch 11, lr 0.00225
Epoch 11: Train 0.2570159168839455, Test 0.25736600160598755
Epoch 12, lr 0.00225
Epoch 12: Train 0.2567605190873146,

KeyboardInterrupt: 